In [4]:
import pandas as pd
import requests
import json
import numpy as np
import dataretrieval as nwis

In [98]:
def getAllStreamStats(i):
    ## Define gage location
#     y_loc = dat["dec_lat_va"][i]
#     x_loc = dat["dec_long_va"][i]
    
    y_loc = np.round(dat["dec_lat_va"][i],2)
    x_loc = np.round(dat["dec_long_va"][i],2)
    state = 'OR'
    gage = dat["gage"][i]
    
    
    ## Define Streamstats API URLS
    # Service URLS
    StreamStatsServiceURLS = {
        'watershed': 'https://streamstats.usgs.gov/streamstatsservices/watershed.geojson',
        'basinCharacteristics': 'https://streamstats.usgs.gov/streamstatsservices/parameters.json',
        'gage': 'https://streamstats.usgs.gov/gagestatsservices/stations'
        }
    NSSServiceURlS = {
        'regressionRegions': 'https://streamstats.usgs.gov/nssservices/regressionregions/bylocation',
        'scenarios': 'https://streamstats.usgs.gov/nssservices/scenarios',
        'computeFlowStats': 'https://streamstats.usgs.gov/nssservices/scenarios/Estimate'
    }

    # Declare cookies for use of consistent StreamStats servers
    global cookies 
    
    # Step 1 - Get the watershed
    print('Step 1 - Get the watershed')
    watershedURLParams = {
        'rcode': state, 
        'xlocation': x_loc, 
        'ylocation': y_loc,
        'crs': 4326,
    #     'crs': 4269,
    }
    delineatedBasin = None
    while delineatedBasin is None: # Sometimes the watershed endpoint returns with no geometry - this loop is written to keep trying until a geometry is returned
        print('Delineating basin...')
        basinDelineationResponse = requests.get(url = StreamStatsServiceURLS['watershed'], params = watershedURLParams)
        if basinDelineationResponse.status_code == 200:
            cookies = basinDelineationResponse.cookies
            try:
                delineatedBasin = json.loads(basinDelineationResponse.content.decode('utf-8'))["featurecollection"][1]["feature"]["features"][0]["geometry"] # this will be used in step 2
                workspaceID = json.loads(basinDelineationResponse.content.decode('utf-8'))["workspaceID"] # this will be used in step 4
            except:
                print("No geometry returned. Retrying...")
                pass
        else:
            print("Error. Retrying...")
            pass

    # Step 2 - Get the Regression Regions associated with the watershed
    print('Step 2 - Get the Regression Regions associated with the watershed')
    regressionRegionPOSTBody = delineatedBasin # from step 1

    regressionRegions = None
    while regressionRegions == None:
        regressionRegionsResponse = requests.post(url = NSSServiceURlS['regressionRegions'], json=regressionRegionPOSTBody)
        if regressionRegionsResponse.status_code == 200:
            regressionRegions = json.loads(regressionRegionsResponse.content.decode('utf-8'))
            regressionRegionCodes = [ sub['code'] for sub in regressionRegions ] # this will be used in step 3
        else:
            print("Error.")
            pass

    # Step 3 - Get the Scenarios associated with the Regression Regions
    print('Step 3 - Get the Scenarios associated with the Regression Regions')

    scenarioURLParams = {
        'regions': state, 
        'statisticgroups': 4, # this can be updated depending on which flow statistics you want to calculate (2 is associated with Peak Flow Statistics) - you can find all statistic groups available for your region at https://streamstats.usgs.gov/docs/nssservices/#/StatisticGroups/GET/StatisticGroups
        'regressionregions': (','.join(regressionRegionCodes)) # from step 2, needs to be comma separated list
    }
    scenarioResponse = requests.get(url = NSSServiceURlS['scenarios'], params=scenarioURLParams)

    parameters = None
    while parameters == None:
        if scenarioResponse.status_code == 200:
            scenarios = json.loads(scenarioResponse.content.decode('utf-8'))[0] # this will be used in step 5
            parameters = json.loads(scenarioResponse.content.decode('utf-8'))[0]["regressionRegions"][0]["parameters"]
            parameterCodes = [ sub['code'] for sub in parameters ] # this will be used in step 4
        else:
            print("Error.")
            pass

    # Step 4 - Compute the basin characteristics
    print('Step 4 - Compute the basin characteristics')
    basinCharacteristicsURLParams = {
        'rcode': state, 
        'workspaceID': workspaceID, # from step 1
        'includeparameters': (','.join(parameterCodes)) # from step 3
    }

    basinCharacteristics = None
    while basinCharacteristics is None:
        basinCharacteristicsResponse = requests.get(url = StreamStatsServiceURLS['basinCharacteristics'], params=basinCharacteristicsURLParams, cookies=cookies)
        if basinCharacteristicsResponse.status_code == 200:
            basinCharacteristics = json.loads(basinCharacteristicsResponse.content.decode('utf-8')) # this will be used in step 5
        else:
            print("Error.")
            pass

    # Step 5 - Compute Flow Statistics
    print('Step 5 - Compute Flow Statistics')
    flowStatsURLParams = {
        'regions': state
    }
    flowStatsPOSTBody = [] 
    for counter, x in enumerate(scenarios['regressionRegions'][0]['parameters']): # from step 3 & step 4
        for p in basinCharacteristics['parameters']:
            if x['code'].lower() == p['code'].lower():
                scenarios['regressionRegions'][0]['parameters'][counter]['value'] = p['value']
    flowStatsPOSTBody.append(scenarios)

    flowStats = None
    while flowStats == None:
        flowStatsResponse = requests.post(url = NSSServiceURlS['computeFlowStats'], params = flowStatsURLParams, json=flowStatsPOSTBody)
        if flowStatsResponse.status_code == 200:
            flowStats = json.loads(flowStatsResponse.content.decode('utf-8'))
        else:
            print("Error.")
            pass

    out = pd.json_normalize(flowStats,
                             record_path=['regressionRegions',['results']],
                            )[["code","value"]]
    out["gage"] = gage
    return out

In [99]:
pnwNPall = pd.read_csv("../data/pnwNPall_InfowStats.csv")
siteID = list(pnwNPall['gage'].astype(str))
dat = nwis.get_info(sites = siteID)[0]


dat["state"] = "none"
dat["state"][dat["state_cd"]==16] = "ID"
dat["state"][dat["state_cd"]==41] = "OR"
dat["state"][dat["state_cd"]==53] = "WA"

st = ["ID",'WA','OR']
dat = dat[dat["state"].isin(st)].reset_index(drop=True)
dat = dat[["site_no","dec_lat_va","dec_long_va",'state','station_nm']]
dat.columns = ["gage","lat","long",'state','name']
dat['gage']  =dat['gage'].astype(int)
dat["lat"] = round(dat["lat"],3)
dat["long"] = round(dat["long"],3)

C:\Users\adamprice\AppData\Local\Temp\ipykernel_43336\1018153480.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dat["state"][dat["state_cd"]==16] = "ID"
C:\Users\adamprice\AppData\Local\Temp\ipykernel_43336\1018153480.py:7: SettingWithC

OverflowError: Python int too large to convert to C long

In [7]:
dat[dat["gage"]==12115700]

,gage,lat,long,state,name


In [117]:
# i = 4


# ## Define gage location
# y_loc = dat["lat"][i]
# x_loc = dat["long"][i]
# state = dat["state"][i]
# gage = dat["gage"][i]


y_loc = 44.3790098
x_loc = -123.5956608
state = 'OR'
gage = 14306100



## Define Streamstats API URLS
# Service URLS
StreamStatsServiceURLS = {
    'watershed': 'https://streamstats.usgs.gov/streamstatsservices/watershed.geojson',
    'basinCharacteristics': 'https://streamstats.usgs.gov/streamstatsservices/parameters.json',
    'gage': 'https://streamstats.usgs.gov/gagestatsservices/stations'
    }
NSSServiceURlS = {
    'regressionRegions': 'https://streamstats.usgs.gov/nssservices/regressionregions/bylocation',
    'scenarios': 'https://streamstats.usgs.gov/nssservices/scenarios',
    'computeFlowStats': 'https://streamstats.usgs.gov/nssservices/scenarios/Estimate'
}

# Declare cookies for use of consistent StreamStats servers
global cookies 

# Step 1 - Get the watershed
print('Step 1 - Get the watershed')
watershedURLParams = {
    'rcode': state, 
    'xlocation': x_loc, 
    'ylocation': y_loc,
    'crs': 4326,
#     'crs': 4269,
}
delineatedBasin = None
while delineatedBasin is None: # Sometimes the watershed endpoint returns with no geometry - this loop is written to keep trying until a geometry is returned
    print('Delineating basin...')
    basinDelineationResponse = requests.get(url = StreamStatsServiceURLS['watershed'], params = watershedURLParams)
    if basinDelineationResponse.status_code == 200:
        cookies = basinDelineationResponse.cookies
        try:
            delineatedBasin = json.loads(basinDelineationResponse.content.decode('utf-8'))["featurecollection"][1]["feature"]["features"][0]["geometry"] # this will be used in step 2
            workspaceID = json.loads(basinDelineationResponse.content.decode('utf-8'))["workspaceID"] # this will be used in step 4
        except:
            print("No geometry returned. Retrying...")
    else:
        print("Error. Retrying...")

# Step 2 - Get the Regression Regions associated with the watershed
print('Step 2 - Get the Regression Regions associated with the watershed')
regressionRegionPOSTBody = delineatedBasin # from step 1

regressionRegions = None
while regressionRegions == None:
    regressionRegionsResponse = requests.post(url = NSSServiceURlS['regressionRegions'], json=regressionRegionPOSTBody)
    if regressionRegionsResponse.status_code == 200:
        regressionRegions = json.loads(regressionRegionsResponse.content.decode('utf-8'))
        regressionRegionCodes = [ sub['code'] for sub in regressionRegions ] # this will be used in step 3
    else:
        print("Error.")
        pass

# Step 3 - Get the Scenarios associated with the Regression Regions
print('Step 3 - Get the Scenarios associated with the Regression Regions')

scenarioURLParams = {
    'regions': state, 
    'statisticgroups': 4, # this can be updated depending on which flow statistics you want to calculate (2 is associated with Peak Flow Statistics) - you can find all statistic groups available for your region at https://streamstats.usgs.gov/docs/nssservices/#/StatisticGroups/GET/StatisticGroups
    'regressionregions': (','.join(regressionRegionCodes)) # from step 2, needs to be comma separated list
}
scenarioResponse = requests.get(url = NSSServiceURlS['scenarios'], params=scenarioURLParams)

parameters = None
while parameters == None:
    if scenarioResponse.status_code == 200:
        scenarios = json.loads(scenarioResponse.content.decode('utf-8'))[0] # this will be used in step 5
        parameters = json.loads(scenarioResponse.content.decode('utf-8'))[0]["regressionRegions"][0]["parameters"]
        parameterCodes = [ sub['code'] for sub in parameters ] # this will be used in step 4
    else:
        print("Error.")
        pass

# Step 4 - Compute the basin characteristics
print('Step 4 - Compute the basin characteristics')
basinCharacteristicsURLParams = {
    'rcode': state, 
    'workspaceID': workspaceID, # from step 1
    'includeparameters': (','.join(parameterCodes)) # from step 3
}

basinCharacteristics = None
while basinCharacteristics is None:
    basinCharacteristicsResponse = requests.get(url = StreamStatsServiceURLS['basinCharacteristics'], params=basinCharacteristicsURLParams, cookies=cookies)
    if basinCharacteristicsResponse.status_code == 200:
        basinCharacteristics = json.loads(basinCharacteristicsResponse.content.decode('utf-8')) # this will be used in step 5
    else:
        print("Error.")
        pass

# Step 5 - Compute Flow Statistics
print('Step 5 - Compute Flow Statistics')
flowStatsURLParams = {
    'regions': state
}
flowStatsPOSTBody = [] 
for counter, x in enumerate(scenarios['regressionRegions'][0]['parameters']): # from step 3 & step 4
    for p in basinCharacteristics['parameters']:
        if x['code'].lower() == p['code'].lower():
            scenarios['regressionRegions'][0]['parameters'][counter]['value'] = p['value']
flowStatsPOSTBody.append(scenarios)

flowStats = None
while flowStats == None:
    flowStatsResponse = requests.post(url = NSSServiceURlS['computeFlowStats'], params = flowStatsURLParams, json=flowStatsPOSTBody)
    if flowStatsResponse.status_code == 200:
        flowStats = json.loads(flowStatsResponse.content.decode('utf-8'))
    else:
        print("Error.")
        pass

out = pd.json_normalize(flowStats,
                         record_path=['regressionRegions',['results']],
                        )[["code","value","equation"]]
out["gage"] = gage

Step 1 - Get the watershed
Delineating basin...
Step 2 - Get the Regression Regions associated with the watershed
Step 3 - Get the Scenarios associated with the Regression Regions
Step 4 - Compute the basin characteristics
Step 5 - Compute Flow Statistics


In [120]:
out.equation[1]

'max(0,1.02287*(10^(-2.7866)*DRNAREA^1.0892*PRECIP^1.6841*WATCAPORR^1.6694))'

In [50]:
i=4

StreamStatsServiceURLS = {
    'gage': 'https://streamstats.usgs.gov/gagestatsservices/characteristics'
    }


gageURLParams = {
    "stationIDOrCode": 12115700
}


gageResponse = requests.get(url = StreamStatsServiceURLS['gage'], params = gageURLParams)

if gageResponse.status_code == 200:
    cookies = gageResponse.cookies
    try:
        gageStats = pd.DataFrame(json.loads(gageResponse.content.decode('utf-8')))


        varCode = pd.DataFrame(list(gageStats["variableType"]))["code"]
        units = pd.DataFrame(list(gageStats["unitType"]))["name"]

        gS = pd.DataFrame(gageStats[["variableTypeID","value"]]).reset_index(drop=True)
        gS["code"] = varCode
        gS["unit"] = units
        gS =gS.set_index('code')
    except:
        pass
    
else:
    pass

In [114]:
METRIC = "M7D10Y"
eq = out['equation'][out["code"]==METRIC].values[0].replace("^", "**")
# TT = eval(eq)
eq

'0.000848*DRNAREA**1.17*PRECIP**1.23'

In [55]:
gageStats

,id,stationID,variableTypeID,unitTypeID,citationID,value,comments,variableType,unitType,citation
0,66800,4789,15,36,19,4.460000,null,"{'id': 15, 'name': '24 Hour 2 Year Precipitati...","{'id': 36, 'name': 'inches', 'abbreviation': '...","{'id': 19, 'title': '2016, Magnitude, frequenc..."
1,66801,4789,18,36,19,7.490000,null,"{'id': 18, 'name': '24 Hour 50 Year Precipitat...","{'id': 36, 'name': 'inches', 'abbreviation': '...","{'id': 19, 'title': '2016, Magnitude, frequenc..."
2,66802,4789,19,36,19,8.140000,null,"{'id': 19, 'name': '24 Hour 100 Year Precipita...","{'id': 36, 'name': 'inches', 'abbreviation': '...","{'id': 19, 'title': '2016, Magnitude, frequenc..."
3,66803,4789,1,35,19,5.068489,null,"{'id': 1, 'name': 'Drainage Area', 'code': 'DR...","{'id': 35, 'name': 'square miles', 'abbreviati...","{'id': 19, 'title': '2016, Magnitude, frequenc..."
4,66804,4789,159,18,19,47.416401,null,"{'id': 159, 'name': 'Latitude of Basin Centroi...","{'id': 18, 'name': 'decimal degrees', 'abbrevi...","{'id': 19, 'title': '2016, Magnitude, frequenc..."
5,66805,4789,160,18,19,-121.756083,null,"{'id': 160, 'name': 'Longitude of Basin Centro...","{'id': 18, 'name': 'decimal degrees', 'abbrevi...","{'id': 19, 'title': '2016, Magnitude, frequenc..."
6,66806,4789,142,41,19,4350.000000,null,"{'id': 142, 'name': 'Maximum Basin Elevation',...","{'id': 41, 'name': 'feet', 'abbreviation': 'ft...","{'id': 19, 'title': '2016, Magnitude, frequenc..."
7,66807,4789,6,41,19,3010.000000,null,"{'id': 6, 'name': 'Mean Basin Elevation', 'cod...","{'id': 41, 'name': 'feet', 'abbreviation': 'ft...","{'id': 19, 'title': '2016, Magnitude, frequenc..."
8,66808,4789,9,1,19,15.630000,null,"{'id': 9, 'name': 'Basin Shape Factor', 'code'...","{'id': 1, 'name': 'dimensionless', 'abbreviati...","{'id': 19, 'title': '2016, Magnitude, frequenc..."
9,66809,4789,4,37,19,-1.690000,null,"{'id': 4, 'name': 'Mean Min January Temperatur...","{'id': 37, 'name': 'degrees F', 'abbreviation'...","{'id': 19, 'title': '2016, Magnitude, frequenc..."


In [52]:
precipCHECK = basinCharacteristics["parameters"][1]["code"]
PRECIP = basinCharacteristics["parameters"][1]["value"]
DRNAREA = gS["value"][gS['code'] == "DRNAREA"].values

KeyError: 'code'

In [75]:

StreamStatsServiceURLS = {
    'gage': 'https://streamstats.usgs.gov/gagestatsservices/stations/'
    }


# gageURLParams = {
#     "idOrCode": 12115700
# }


gageResponse = requests.get(url = StreamStatsServiceURLS['gage']+'12115700')

if gageResponse.status_code == 200:
    cookies = gageResponse.cookies
    try:
        gageStats = pd.DataFrame(json.loads(gageResponse.content.decode('utf-8')))


#         varCode = pd.DataFrame(list(gageStats["variableType"]))["code"]
#         units = pd.DataFrame(list(gageStats["unitType"]))["name"]

#         gS = pd.DataFrame(gageStats[["variableTypeID","value"]]).reset_index(drop=True)
#         gS["code"] = varCode
#         gS["unit"] = units
#         gS =gS.set_index('code')
    except:
        pass
    
else:
    pass

In [78]:
gageResponse

<Response [200]>

In [93]:
json.loads(gageResponse.content.decode('utf-8'))["region"]['id']

51

In [77]:
gageStats

,id,code,agencyID,name,isRegulated,stationTypeID,location,regionID,stationType
0,3,EC_05PD028,3,LAKE 661 OUTLET NEAR KENORA,False,1,"{'type': 'Point', 'coordinates': [-93.7397, 49...",67,"{'id': 1, 'name': 'Gaging Station, continuous ..."
1,4,EC_05PD024,3,LAKE 239 LOWER EAST INLET NEAR KENORA,False,1,"{'type': 'Point', 'coordinates': [-93.7133, 49...",67,"{'id': 1, 'name': 'Gaging Station, continuous ..."
2,5,EC_05PD023,3,LAKE 239 OUTLET NEAR KENORA,False,1,"{'type': 'Point', 'coordinates': [-93.7267, 49...",67,"{'id': 1, 'name': 'Gaging Station, continuous ..."
3,8,EC_05PD017,3,LAKE 470 OUTLET NEAR KENORA,False,1,"{'type': 'Point', 'coordinates': [-93.7325, 49...",67,"{'id': 1, 'name': 'Gaging Station, continuous ..."
4,9,EC_05PD015,3,LAKE 240 OUTLET NEAR KENORA,False,1,"{'type': 'Point', 'coordinates': [-93.7261, 49...",67,"{'id': 1, 'name': 'Gaging Station, continuous ..."
5,13,EC_05PC010,3,STURGEON RIVER NEAR BARWICK,False,1,"{'type': 'Point', 'coordinates': [-93.9833, 48...",67,"{'id': 1, 'name': 'Gaging Station, continuous ..."
6,14,EC_05PB018,3,ATIKOKAN RIVER AT ATIKOKAN,False,1,"{'type': 'Point', 'coordinates': [-91.5841, 48...",67,"{'id': 1, 'name': 'Gaging Station, continuous ..."
7,16,EC_05PB014,3,TURTLE RIVER NEAR MINE CENTRE,False,1,"{'type': 'Point', 'coordinates': [-92.7238, 48...",67,"{'id': 1, 'name': 'Gaging Station, continuous ..."
8,17,50345000,1,"JOLLY HILL GUT AT JOLLY HILL, ST. CROIX USVI",False,1,"{'type': 'Point', 'coordinates': [-64.8626461,...",64,"{'id': 1, 'name': 'Gaging Station, continuous ..."
9,18,50332000,1,"RIVER GUT AT RIVER, ST. CROIX USVI",False,1,"{'type': 'Point', 'coordinates': [-64.8140344,...",64,"{'id': 1, 'name': 'Gaging Station, continuous ..."
